<a href="https://colab.research.google.com/github/Abass-AIM/Artificial-Intelligence/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Remove broken TensorFlow packages
!pip uninstall -y tensorflow tf-nightly tensorflow-cpu tensorflow-intel

# 2️⃣ Install a stable compatible version
!pip install tensorflow==2.17.0 tensorflow-datasets


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print("TensorFlow version:", tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from tensorflow.keras import layers
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Map labels to 0 (ham) and 1 (spam)
label_map = {'ham': 0, 'spam': 1}
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)

#Split data
X_train = train_df['message'].values
y_train = train_df['label'].values
X_test = test_df['message'].values
y_test = test_df['label'].values


# Text vectorization
max_vocab_size = 5000
max_len = 100  # max number of words per message

vectorizer = keras.layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=max_len
)

# Adapt vectorizer to training messages
vectorizer.adapt(X_train)

#build model
model = keras.Sequential([
    vectorizer,
    layers.Embedding(input_dim=max_vocab_size, output_dim=64, mask_zero=True),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)


In [ ]:
def predict_message(message):
    """
    Returns [probability, label]
    probability: float between 0 and 1 (spam likelihood)
    label: "ham" or "spam"
    """
    # Wrap message in a tensor of dtype string
    input_tensor = tf.constant([message], dtype=tf.string)
    prob = model.predict(input_tensor, verbose=0)[0][0]
    label = "spam" if prob >= 0.5 else "ham"
    return [float(prob), label]

# Test examples
print(predict_message("Congratulations! You have won a free ticket. Call now!"))
print(predict_message("Hey, are we meeting for lunch today?"))


In [ ]:
def predict_message(pred_text):
    """
    Predict whether a message is ham or spam.
    Returns [probability, label], e.g. [0.0083, 'ham']
    """
    # Convert input text to a TensorFlow string tensor
    input_tensor = tf.constant([pred_text], dtype=tf.string)

    # Predict probability (output of sigmoid)
    prob = model.predict(input_tensor, verbose=0)[0][0]

    # Determine label based on probability
    label = "spam" if prob >= 0.5 else "ham"

    return [float(prob), label]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
